In [1]:
import pandas as pd

import plotly.express as px

#import warnings

#warnings.simplefilter('ignore')

In [3]:
gastos = pd.read_csv(
    'https://www.gov.br/secretariageral/pt-br/acesso-a-informacao/informacoes-classificadas-e-desclassificadas/Planilha12003a2022.csv',
    encoding='latin1', delimiter=';')

In [4]:
gastos_processed = gastos.copy()
gastos_processed

,DATA PGTO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC
0,02/01/2003,***.010.008-**,3.134920e+13,JW TRANSPORTADORA TURÖSTICA,"R$ 1.696,90",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N
1,02/01/2003,***.010.008-**,3.134920e+13,JW TRANSPORTADORA TURÖSTICA,"R$ 1.336,60",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N
2,02/01/2003,***.868.251-**,3.134920e+13,JW TRANSPORTADORA TURÖSTICA,"R$ 1.396,43",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.600539/2003-63.R.08.18/12/2002.31/12/2014.N
3,03/01/2003,***.004.131-**,5.927170e+11,FLORES ALVORADA,"R$ 8.585,00",C,OUTROS SERVIOS DE TERCEIROS-PESSOA JURÖDICA,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N
4,03/01/2003,***.004.131-**,8.393080e+11,CRISTAL LIMPEZA - COMRCIO DE MATERIAIS PARA L...,"R$ 251,50",C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAÇO,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N
...,...,...,...,...,...,...,...,...
113337,04/12/2022,***.136.591-**,4.750841e+13,COMPANHIA BRASILEIRA DE DISTRIBUIÇO,"518,07",C,GENEROS DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N
113338,04/12/2022,***.136.591-**,1.255044e+13,RESTAURANTE SOHO,"330,52",C,FORNECIMENTO DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N
113339,04/12/2022,***.136.591-**,3.696869e+12,BIG TRANS COMERCIAL DE ALIMENTOS LTDA,"54,66",C,GENEROS DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N
113340,Fonte: SUPRIM,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
gastos_processed.loc[:, 'DATA PGTO'] = pd.to_datetime(gastos_processed['DATA PGTO'], errors='coerce', format="%d/%m/%Y")

gastos_processed.loc[:, 'ANO'] = gastos_processed['DATA PGTO'].dt.year


gastos_processed = gastos_processed.dropna(axis=0, subset=['CPF SERVIDOR'])

gastos_processed.loc[:, 'VALOR_NUM'] = pd.to_numeric(gastos_processed['VALOR'].str.replace(
    r'R\$', '').str.replace(".", '').str.replace(',', '.').str.strip(), errors='coerce')

gastos_processed = gastos_processed.dropna(subset=['VALOR_NUM'])

gastos_processed

/tmp/ipykernel_8535/4023865649.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  gastos_processed.loc[:, 'DATA PGTO'] = pd.to_datetime(gastos_processed['DATA PGTO'], errors='coerce', format="%d/%m/%Y")
/tmp/ipykernel_8535/4023865649.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  gastos_processed.loc[:, 'VALOR_NUM'] = pd.to_numeric(gastos_processed['VALOR'].str.replace(
/tmp/ipykernel_8535/4023865649.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  gastos_processed.loc[:, 'VALOR_NUM'] = pd.to_numeric(gastos_processed['VALOR'].str.replace(
/tmp/ipykernel_8

,DATA PGTO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC,ANO,VALOR_NUM
0,2003-01-02,***.010.008-**,3.134920e+13,JW TRANSPORTADORA TURÖSTICA,"R$ 1.696,90",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,2003.0,1696.90
1,2003-01-02,***.010.008-**,3.134920e+13,JW TRANSPORTADORA TURÖSTICA,"R$ 1.336,60",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,2003.0,1336.60
2,2003-01-02,***.868.251-**,3.134920e+13,JW TRANSPORTADORA TURÖSTICA,"R$ 1.396,43",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.600539/2003-63.R.08.18/12/2002.31/12/2014.N,2003.0,1396.43
3,2003-01-03,***.004.131-**,5.927170e+11,FLORES ALVORADA,"R$ 8.585,00",C,OUTROS SERVIOS DE TERCEIROS-PESSOA JURÖDICA,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,2003.0,8585.00
4,2003-01-03,***.004.131-**,8.393080e+11,CRISTAL LIMPEZA - COMRCIO DE MATERIAIS PARA L...,"R$ 251,50",C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAÇO,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,2003.0,251.50
...,...,...,...,...,...,...,...,...,...,...
113335,2022-12-03,***.136.591-**,1.726166e+13,OUTBACK STEAKHOUSE RESTAURANTES BRASIL S/A,"179,5",C,FORNECIMENTO DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,2022.0,179.50
113336,2022-12-03,***.136.591-**,3.987994e+13,BIOMUNDO 105 NORTE COMERCIO DE ALIMENTOS,"54,95",C,GENEROS DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,2022.0,54.95
113337,2022-12-04,***.136.591-**,4.750841e+13,COMPANHIA BRASILEIRA DE DISTRIBUIÇO,"518,07",C,GENEROS DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,2022.0,518.07
113338,2022-12-04,***.136.591-**,1.255044e+13,RESTAURANTE SOHO,"330,52",C,FORNECIMENTO DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,2022.0,330.52


In [6]:
def presidente(ano):

    if ano <= 2010:
        return 'LULA'
    elif ano <= 2016:
        return 'DILMA'
    elif ano <= 2018:
        return "TEMER"
    elif ano <= 2022:
        return "BOLSONARO"


gastos_processed['PRES'] = gastos_processed['ANO'].apply(presidente)
gastos_processed

,DATA PGTO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC,ANO,VALOR_NUM,PRES
0,2003-01-02,***.010.008-**,3.134920e+13,JW TRANSPORTADORA TURÖSTICA,"R$ 1.696,90",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,2003.0,1696.90,LULA
1,2003-01-02,***.010.008-**,3.134920e+13,JW TRANSPORTADORA TURÖSTICA,"R$ 1.336,60",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,2003.0,1336.60,LULA
2,2003-01-02,***.868.251-**,3.134920e+13,JW TRANSPORTADORA TURÖSTICA,"R$ 1.396,43",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.600539/2003-63.R.08.18/12/2002.31/12/2014.N,2003.0,1396.43,LULA
3,2003-01-03,***.004.131-**,5.927170e+11,FLORES ALVORADA,"R$ 8.585,00",C,OUTROS SERVIOS DE TERCEIROS-PESSOA JURÖDICA,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,2003.0,8585.00,LULA
4,2003-01-03,***.004.131-**,8.393080e+11,CRISTAL LIMPEZA - COMRCIO DE MATERIAIS PARA L...,"R$ 251,50",C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAÇO,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,2003.0,251.50,LULA
...,...,...,...,...,...,...,...,...,...,...,...
113335,2022-12-03,***.136.591-**,1.726166e+13,OUTBACK STEAKHOUSE RESTAURANTES BRASIL S/A,"179,5",C,FORNECIMENTO DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,2022.0,179.50,BOLSONARO
113336,2022-12-03,***.136.591-**,3.987994e+13,BIOMUNDO 105 NORTE COMERCIO DE ALIMENTOS,"54,95",C,GENEROS DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,2022.0,54.95,BOLSONARO
113337,2022-12-04,***.136.591-**,4.750841e+13,COMPANHIA BRASILEIRA DE DISTRIBUIÇO,"518,07",C,GENEROS DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,2022.0,518.07,BOLSONARO
113338,2022-12-04,***.136.591-**,1.255044e+13,RESTAURANTE SOHO,"330,52",C,FORNECIMENTO DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,2022.0,330.52,BOLSONARO


In [9]:
#gastos_processed = gastos_processed.sort_values("ANO")
#gastos_processed

,DATA PGTO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC,ANO,VALOR_NUM,PRES
0,2003-01-02,***.010.008-**,3.134920e+13,JW TRANSPORTADORA TURÖSTICA,"R$ 1.696,90",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,2003.0,1696.90,LULA
3250,2003-08-22,***.004.131-**,4.750841e+13,COMPANHIA BRASILEIRA DE DISTRIBUÖÇO - EXTRA,"R$ 29,54",C,MATERIAL ELETRICO E ELETRONICO,00200.803806/2003-54.R.08.20/08/2003.31/12/2014.N,2003.0,29.54,LULA
3249,2003-08-22,***.004.131-**,4.750841e+13,COMPANHIA BRASILEIRA DE DISTRIBUÖÇO - EXTRA,"R$ 120,00",C,GENEROS DE ALIMENTAÇO,00200.803806/2003-54.R.08.20/08/2003.31/12/2014.N,2003.0,120.00,LULA
3248,2003-08-22,***.765.178-**,4.328381e+13,KALUNGA COM E IND. GRµFICA LTDA.,"R$ 25,99",C,MATERIAL DE EXPEDIENTE,00200.003808/2003-43.R.08.20/08/2003.31/12/2014.N,2003.0,25.99,LULA
3247,2003-08-22,***.570.341-**,6.241388e+13,MELIµ BRASIL ADM. HOTELEIRA COMERCIAL LTDA,"R$ 583,25",D,HOSPEDAGENS,00200.203803/2003-11.R.08.18/08/2003.31/12/2014.N,2003.0,583.25,LULA
...,...,...,...,...,...,...,...,...,...,...,...
111315,2022-04-21,***.357.911-**,4.453855e+13,BEIRA MAR COMERCIO DE COMBUSTIVEIS E LUBRIFICA...,"86,63",C,COMBUSTIVEIS E LUBRIF. AUTOMOTIVOS,00150.000221/2022-88.R.08.23/03/2022.31/12/2022.N,2022.0,86.63,BOLSONARO
111316,2022-04-21,***.357.911-**,4.453855e+13,BEIRA MAR COMERCIO DE COMBUSTIVEIS E LUBRIFICA...,"150,09",C,COMBUSTIVEIS E LUBRIF. AUTOMOTIVOS,00150.000221/2022-88.R.08.23/03/2022.31/12/2022.N,2022.0,150.09,BOLSONARO
111317,2022-04-21,***.357.911-**,4.453855e+13,BEIRA MAR COMERCIO DE COMBUSTIVEIS E LUBRIFICA...,81,C,COMBUSTIVEIS E LUBRIF. AUTOMOTIVOS,00150.000221/2022-88.R.08.23/03/2022.31/12/2022.N,2022.0,81.00,BOLSONARO
111308,2022-04-22,***.357.911-**,5.846287e+12,CAMBU¨ SUPERMERCADOS LTDA,"375,48",C,GENEROS DE ALIMENTAÇO,00150.000221/2022-88.R.08.23/03/2022.31/12/2022.N,2022.0,375.48,BOLSONARO


In [7]:
gastos_presidente = gastos_processed.groupby(["ANO", "PRES"]).agg({"VALOR_NUM": sum}).reset_index()
gastos_presidente

,ANO,PRES,VALOR_NUM
0,2003.0,LULA,5327599.63
1,2004.0,LULA,6541238.37
2,2005.0,LULA,5204035.80
3,2006.0,LULA,4977941.73
4,2007.0,LULA,3857905.40
5,2008.0,LULA,6084616.50
6,2009.0,LULA,5438905.30
7,2010.0,LULA,6557394.19
8,2011.0,DILMA,4587842.43
9,2012.0,DILMA,4598570.59


In [8]:
fig = px.bar(gastos_presidente, x='ANO', y="VALOR_NUM", color='PRES',
             title="Gastos do cartão presidencial por presidente",
             labels={"VALOR_NUM": "Valor em milhões", 'PRES': "PRESIDENTE"})

fig = fig.update_xaxes(tickangle=45, tickvals=gastos_presidente['ANO'][::])

In [9]:
fig.show()

In [47]:
lula_mandato_1 = gastos_presidente.loc[0:3].groupby(["PRES"]).agg({"VALOR_NUM": sum}).reset_index()
lula_mandato_1['PRES'] = 'LULA_1'

lula_mandato_2 = gastos_presidente.loc[4:7].groupby(["PRES"]).agg({"VALOR_NUM": sum}).reset_index()
lula_mandato_2['PRES'] = 'LULA_2'

dilma_mandato_1 = gastos_presidente.loc[8:11].groupby(["PRES"]).agg({"VALOR_NUM": sum}).reset_index()
dilma_mandato_1['PRES'] = 'DILMA_1'

dilma_mandato_2 = gastos_presidente.loc[12:13].groupby(["PRES"]).agg({"VALOR_NUM": sum}).reset_index()
dilma_mandato_2['PRES'] = 'DILMA_2'

temer = gastos_presidente.loc[14:15].groupby(["PRES"]).agg({"VALOR_NUM": sum}).reset_index()
bolsonaro = gastos_presidente.loc[16:].groupby(["PRES"]).agg({"VALOR_NUM": sum}).reset_index()
bolsonaro

gastos_acumulados = pd.concat([lula_mandato_1, lula_mandato_2, dilma_mandato_1, dilma_mandato_2, temer, bolsonaro], axis=0, ignore_index=True)
gastos_acumulados


,PRES,VALOR_NUM
0,LULA_1,22050815.53
1,LULA_2,21938821.39
2,DILMA_1,24513516.26
3,DILMA_2,9899205.36
4,TEMER,8869681.44
5,BOLSONARO,27621657.23


In [51]:
fig_acumulado = px.bar(gastos_acumulados, x='PRES', y='VALOR_NUM', color='PRES', title='Gastos do cartão presidencial por tipo',
labels={"VALOR_NUM": "Valor em milhões", 'PRES': "Presidente"})

fig_acumulado.show()

In [10]:
gastos_lula = gastos_processed[gastos_processed['PRES'] == 'LULA']
gastos_lula

,DATA PGTO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC,ANO,VALOR_NUM,PRES
0,2003-01-02,***.010.008-**,3.134920e+13,JW TRANSPORTADORA TURÖSTICA,"R$ 1.696,90",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,2003.0,1696.90,LULA
1,2003-01-02,***.010.008-**,3.134920e+13,JW TRANSPORTADORA TURÖSTICA,"R$ 1.336,60",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,2003.0,1336.60,LULA
2,2003-01-02,***.868.251-**,3.134920e+13,JW TRANSPORTADORA TURÖSTICA,"R$ 1.396,43",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.600539/2003-63.R.08.18/12/2002.31/12/2014.N,2003.0,1396.43,LULA
3,2003-01-03,***.004.131-**,5.927170e+11,FLORES ALVORADA,"R$ 8.585,00",C,OUTROS SERVIOS DE TERCEIROS-PESSOA JURÖDICA,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,2003.0,8585.00,LULA
4,2003-01-03,***.004.131-**,8.393080e+11,CRISTAL LIMPEZA - COMRCIO DE MATERIAIS PARA L...,"R$ 251,50",C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAÇO,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,2003.0,251.50,LULA
...,...,...,...,...,...,...,...,...,...,...,...
65865,2010-12-30,***.570.341-**,7.653576e+13,BRASIL TELECOM S/A - FILIAL CRT,"R$ 455,73",D,SERVIOS DE TELECOMUNICAåES,00150.001812/2010-39.R.08.27/11/2010.31/12/2014.N,2010.0,455.73,LULA
65866,2010-12-31,***.262.471-**,1.121094e+13,VIANDE BOUTIQUE DE CARNES LTDA ME,"R$ 224,54",C,GENEROS DE ALIMENTAÇO,00150.001802/2010-01.R.08.27/11/2010.31/12/2014.N,2010.0,224.54,LULA
65867,2010-12-31,***.262.471-**,3.841910e+12,NAZAR TECIDOS E PLµSTICOS,"R$ 1.550,00",C,MATERIAL DE COPA E COZINHA,00150.001802/2010-01.R.08.27/11/2010.31/12/2014.N,2010.0,1550.00,LULA
65868,2010-12-31,***.262.471-**,1.610583e+12,DISTRIBUIDORA DE CARNE MORRO BRANCO LTDA - ME,"R$ 313,56",C,GENEROS DE ALIMENTAÇO,00150.001802/2010-01.R.08.27/11/2010.31/12/2014.N,2010.0,313.56,LULA


In [12]:
gastos_lula_setores = gastos_lula.groupby(["TIPO", "SUBELEMENTO DE DESPESA"]).agg({"VALOR_NUM": sum}).reset_index()
gastos_lula_setores

,TIPO,SUBELEMENTO DE DESPESA,VALOR_NUM
0,C,ALIMENTOS PARA ANIMAIS,50659.31
1,C,APARELHOS E UTENSÖLIOS DOMSTICOS,288.00
2,C,ASSINATURAS DE PERIàDICOS E ANUIDADES,37.10
3,C,BANDEIRAS FLAMULAS E INSIGNAS,80.00
4,C,COMBUSTIVEIS E LUBRF. P/ OUTRAS FINALIDADES,4707.06
...,...,...,...
158,D,SERVIOS GRµFICOS,5743.20
159,D,SERVIOS JUDICIµRIOS,2.65
160,D,SOBRESSALENTES DE ARMAMENTO,4756.19
161,D,"UNIFORMES, TECIDOS E AVIAMENTOS",4835.65


In [13]:
gastos_lula_setores = gastos_processed.groupby(["TIPO"]).agg({"VALOR_NUM": sum}).reset_index()
gastos_lula_setores

,TIPO,VALOR_NUM
0,C,1.036233e+08
1,D,1.127038e+07


In [60]:
fig_lula = px.bar(gastos_lula_setores, x='TIPO', y="VALOR_NUM", color='TIPO',
             title="Gastos do cartão presidencial por tipo",
             labels={"VALOR_NUM": "Valor em milhões", 'TIPO': "Tipo do gasto"},
             color_discrete_sequence=px.colors.sequential.Greens_r)

In [61]:
fig_lula.show()

In [53]:

fig_lula = px.pie(gastos_lula, values='VALOR_NUM', names='TIPO', title='Gastos do cartão presidencial por tipo',
color_discrete_sequence=px.colors.sequential.RdBu)
fig_lula.show()

,Unidade,UG,ND,Detalhamento ND,fev-22,mar-22,abr-22,mai-22,jun-22,jul-22,ago-22,set-22,out-22,nov-22,dez-22,Total
0,SA/SG-PR,110001.0,33903022.0,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,"4946,1","3344,31",NaN,"12439,03","5523,25","6334,71","6182,28","7539,69","12988,38","4382,1","12166,65","75846,5"
1,SA/SG-PR,110001.0,33903941.0,FORNECIMENTO DE ALIMENTACAO,"13168,8","163339,58","164334,25","286343,88","271991,26","325456,41","558095,95","541551,91","391737,43","525600,02","242005,13","3483624,62"
2,SA/SG-PR,110001.0,33903007.0,GENEROS DE ALIMENTACAO,"55803,86","69126,89",2053,"155074,25","84743,85","179318,54","94838,67","181123,34",95612,"114565,49","146337,48","1178597,37"
3,SA/SG-PR,110001.0,33903680.0,HOSPEDAGENS PF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6400,NaN,NaN,NaN,6400
4,SA/SG-PR,110001.0,33903980.0,HOSPEDAGENS PJ,"78819,85","357445,8","162884,29","834588,81","382874,45","639264,62","576299,55","670177,68","504108,3","591499,43","346950,27","5144913,05"
5,SA/SG-PR,110001.0,33903910.0,LOCACAO DE IMOVEIS,1350,NaN,2580,NaN,3800,6300,NaN,8005,1848,11087,NaN,34970
6,SA/SG-PR,110001.0,33903912.0,LOCACAO DE MAQUINAS E EQUIPAMENTOS,NaN,8500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8500
7,NaN,NaN,33903914.0,LOCACAO BENS MOV. OUT.NATUREZAS E INTANGIVEIS,NaN,60740,41500,"28429,99",40100,"61189,99","53034,79",74200,62300,45855,16100,"483449,77"
8,SA/SG-PR,110001.0,33903979.0,"SERV. DE APOIO ADMIN., TECNICO E OPERACIONAL","17833,49","163136,29","547818,62","42369,79","44380,72","71412,62","166477,42","293073,74","593270,09","48456,3","85010,03","2073239,11"
9,SA/SG-PR,110001.0,33903001.0,COMBUSTIVEIS E LUBRIFICANTES AUTOMOTIVOS,NaN,"85665,34","9338,03","54356,37","1633,71","15384,35","23045,84","16984,1",2195,"9323,8","4261,59","222188,13"
